In [416]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc

In [376]:
df = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/NCCR_combined_corpus_DE_wording_available_TEST.csv")

In [394]:
nlp = spacy.load("de_core_news_lg", exclude=['tok2vec', 'tagger', 'morphologizer', 'parser',
                         'attribute_ruler', 'lemmatizer'])

nlp.add_pipe("sentencizer")



In [395]:
def standardize_text(text):
    # Replace special characters #todo: simplify replacement
    text = text.replace("ä", "ae").replace("ü", "ue").replace("ö", "oe").replace("Ö", "Oe").replace("Ä", "Ae").replace("Ü", "Ue").replace("ß", "ss").replace("@", "").replace("/", "")
    text = text.replace("ç", "c"). replace("", "")
    
    text = text.lower()
    text = " ".join(text.split())

    return text

In [396]:
def standardize_wording(wording):
    # Replace special characters #todo: simplify replacement
    wording = wording.replace("/", "").replace("@", "").replace("<ord:65430>", "")
    wording = " ".join(wording.split())

    return wording

In [397]:
df['text_prep'] = df['text_prep'].apply(lambda x: standardize_text(x))

In [398]:
df['Wording'] = df['Wording'].apply(lambda x: standardize_wording(x))

In [400]:
df['text_prep'] 

0      "bucher fordert ehrenerklaerung von allen parl...
1      na berlin (ots) - der rheinland-pfaelzische fi...
2      wallner: gruene transparenz-offensive soll kla...
3      zum geheimdienstskandal weltweit umfassender a...
4      "genug gezahlt - ueberstunden steuerfrei", "jo...
                             ...                        
944    na berlin (ots) - buendnis 90die gruenen haben...
945    na berlin. vor der parlamentswahl in simbabwe ...
946    na muenchen die 600 plaetze der festhalle des ...
947    bilaterale ja eu-beitritt nein. dank den bilat...
948    na bemaengelte oekonomisierung von ehe und fam...
Name: text_prep, Length: 949, dtype: object

In [401]:
df['Wording'] 

0      bucher fordert ehrenerklaerung von allen parla...
1      sein parteikollege, der rheinland-pfaelzische ...
2      das team stronach hat bis april 2013 laut rech...
3                                                    wir
4      waehrend andere parteien mit nackter haut vers...
                             ...                        
944                                              gruenen
945    die fdp-bundestagsfraktion fordert alle staatl...
946                                      die altparteien
947                                                  svp
948                                        norbert bluem
Name: Wording, Length: 949, dtype: object

In [425]:
df['doc'] = list(nlp.pipe(df['text_prep']))
df['wording_doc'] = list(nlp.pipe(df['Wording']))




In [424]:
doc = nlp('Das ist ein Test der das Ganze prüft')
get_sub_wording(doc)

Das ist ein Test der 

In [436]:
def get_sub_wording(wording):
    doc2 = Doc(wording.vocab, words=[t.text for i, t in enumerate(wording) if i <3])
    return doc2 
    


In [437]:
df['wording_doc'] = df['wording_doc'].apply(lambda x: get_sub_wording(x))

In [438]:
df['wording_doc']

0          (bucher, fordert, ehrenerklaerung)
1                    (sein, parteikollege, ,)
2                       (das, team, stronach)
3                                       (wir)
4                (waehrend, andere, parteien)
                        ...                  
944                                 (gruenen)
945    (die, fdp-bundestagsfraktion, fordert)
946                        (die, altparteien)
947                                     (svp)
948                          (norbert, bluem)
Name: wording_doc, Length: 949, dtype: object

In [439]:
# Define function to find index of tokens that match Wording content
def get_matches(doc, wording):
    
    # Define Spacy Matcher
    matcher = PhraseMatcher(nlp.vocab)
    # Add patterns from nlp-preprocessed Wording column
    matcher.add("WORDING", [wording])
    
    # Get matches
    matches = matcher(doc)

    return matches



In [440]:
df['matches_new'] = df.apply(lambda x: get_matches(x['doc'], x['wording_doc']), axis=1)

## Fix non-matches

In [441]:
df['matches_new']

0                          [(1212883764967918647, 1, 4)]
1                      [(1212883764967918647, 345, 348)]
2      [(1212883764967918647, 791, 794), (12128837649...
3      [(1212883764967918647, 29, 30), (1212883764967...
4                        [(1212883764967918647, 60, 63)]
                             ...                        
944    [(1212883764967918647, 8, 9), (121288376496791...
945                      [(1212883764967918647, 68, 71)]
946                    [(1212883764967918647, 117, 119)]
947                      [(1212883764967918647, 97, 98)]
948                      [(1212883764967918647, 19, 21)]
Name: matches_new, Length: 949, dtype: object

In [442]:
non = df[~df['matches_new'].astype(bool)]
non = non[['ID', 'doc', 'Wording', 'matches_new', 'wording_segments']]
#non['wording_doc'] = list(nlp.pipe(non['Wording']))
non['doc_tokens'] = non['doc'].apply(lambda x: [token.text for token in x])
non['wording_tokens'] = non['wording_doc'].apply(lambda x: [token.text for token in x])

In [443]:
len(non)

47

In [444]:
non['Wording'].iloc[1]

'volksparte'

In [446]:
non['doc'].iloc[1]

1.6. oesterreich und europa. europaeische union. die volkspartei sieht in der zukuenftigen mitgliedschaft oesterreichs zur europaeischen union die wesentlichste voraussetzung zum ausbau unserer, wirtschaftskraft und zur sicherung der beschaeftigung in oesterreich. der zielstrebigkeit der volkspartei ist es zu verdanken, dass oesterreich am 17. juli 1989 den antrag auf beitritt zu den europaeischen gemeinschaften gestellt hat. am 12. juni 1994 haben die oesterreicherinnen und oesterreicher mit ueberwaeltigender mehrheit entschieden, dass oesterreich mit beginn des kommenden jahres der europaeischen union als gleichberechtigter partner angehoeren wird. damit wurde fuer oesterreich die wichtigste entscheidung seit dem abschluss des staatsvertrag zum wohle unserer heimat getroffen. die antwort der europaeischen union auf die weltweiten herausforderungen, vor allem im technischen und wirtschaftlichen bereich, sind der binnenmarkt und die einheitliche europaeische akte sowie deren weiterentw

In [447]:
print(str(non.iloc[2]['doc_tokens']))

['liberal-sozial', ':-)']


In [448]:
print(str(non.iloc[2]['wording_tokens']))

['barbara', 'schmid-federer']


In [ ]:
--> try match first 5 tokens

## Investigate Missing Segments

In [4]:
# sample type?
import pandas as pd
non_train = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TRAIN.csv")
#non_test = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/non_matched_TEST.csv")

In [5]:
non_train

,Unnamed: 0,level_0,index,ID,text,Coder,#TS,#TS2,Codierstart_Lab,Medium,...,year,text_temp,Wording_temp,doc_temp,Wording_doc_temp,wording_matches,wording_sentence,wording_segments,doc_tokens,wording_tokens
0,462,462,418,au_pm_el_94_50027.txt,Text ID:\nau_pm_el_94_50027.txt\n\nParty Manif...,pagr,1.405505e+09,1.405506e+09,Wed Jul 16 12:01:49 2014,1160,...,1994,DER STAAT DER BUeRGERINNEN UND BUeRGER Wir wer...,"Wir werden dafür sorgen, dass der Staat möglic...",DER STAAT DER BUeRGERINNEN UND BUeRGER Wir wer...,"Wir werden dafür sorgen ,",[],[],NaN,"['DER', 'STAAT', 'DER', 'BUeRGERINNEN', 'UND',...","['Wir', 'werden', 'dafür', 'sorgen', ',']"
1,78,78,93,au_pm_el_02_40040.txt,Text ID:\nau_pm_el_02_40040.txt\n\nParty Manif...,lire,1.405091e+09,1.405092e+09,Fri Jul 11 16:57:35 2014,1160,...,2002,Gesundheit Gesund werden ? gesund bleiben Die ...,Für uns Christdemokraten steht der Mensch im M...,Gesundheit Gesund werden ? gesund bleiben Die ...,Für uns Christdemokraten steht der,[],[],NaN,"['Gesundheit', 'Gesund', 'werden', '?', 'gesun...","['Für', 'uns', 'Christdemokraten', 'steht', 'd..."
2,44,44,36,au_pm_el_02_10038.txt,Text ID:\nau_pm_el_02_10038.txt\n\nParty Manif...,elan,1.404674e+09,1.404677e+09,Sun Jul 06 21:07:08 2014,1160,...,2002,TRANSIT. DAS ERBE. Der zu Zeiten der grossen K...,Darüber hinaus wurde vom Verkehrsministerium e...,TRANSIT. DAS ERBE. Der zu Zeiten der grossen K...,Darüber hinaus wurde vom Verkehrsministerium,[],[],NaN,"['TRANSIT', '.', 'DAS', 'ERBE', '.', 'Der', 'z...","['Darüber', 'hinaus', 'wurde', 'vom', 'Verkehr..."
3,3397,3397,2127,cd_tw_db_13043_00176.txt,Text ID:\ncd_tw_db_13043_00176.txt\n\n\nTweet ...,difr,1.411630e+09,1.411630e+09,Thu Sep 25 09:31:04 2014,9901,...,2014,Hallo? @GuyVerhofstadt fordert mehr private Mo...,GuyVerhofstadt,Hallo? @GuyVerhofstadt fordert mehr private Mo...,GuyVerhofstadt,[],[],NaN,"['Hallo', '?', '@GuyVerhofstadt', 'fordert', '...",['GuyVerhofstadt']
4,3004,3004,1646,cd_pm_el_83_70017.txt,Text ID:\ncd_pm_el_83_70017.txt\n\nParty Manif...,kryo,1.405259e+09,1.405260e+09,Sun Jul 13 15:48:29 2014,1360,...,1983,Finanzplatz 1. Die Schweiz ist ein rohstoffarm...,Die SVP ist über die in- und ausländischen Bes...,Finanzplatz 1. Die Schweiz ist ein rohstoffarm...,Die SVP ist über die,[],[],NaN,"['Finanzplatz', '1.', 'Die', 'Schweiz', 'ist',...","['Die', 'SVP', 'ist', 'über', 'die']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,3482,3482,2376,de_fb_db_15007_00023z.txt,Text ID:\nde_fb_db_15007_00023.txt\n\n\nFacebo...,mabu,1.464203e+09,1.464205e+09,Wed May 25 21:01:00 2016,9902,...,2015,#Asyl Das Grundgesetz ist die gemeinsame Grund...,Wir brauchen mehr <ORD:65412>Anpackgesellschaf...,#Asyl Das Grundgesetz ist die gemeinsame Grund...,Wir brauchen mehr < ORD:65412>Anpackgesellschaft,[],[],NaN,"['#', 'Asyl', 'Das', 'Grundgesetz', 'ist', 'di...","['Wir', 'brauchen', 'mehr', '<', 'ORD:65412>An..."
558,4095,4093,3060,de_pm_el_94_30001.txt,Text ID:\nde_pm_el_94_30001.txt\n\nParty Manif...,pagr,1.408525e+09,1.408526e+09,Wed Aug 20 10:52:58 2014,1560,...,1994,BUeNDNIS 90DIE GRUeNEN 41113.1994 Programm zur...,BÜNDNIS 90DIE GRÜNEN treten zu den Wahlen zum ...,BUeNDNIS 90DIE GRUeNEN 41113.1994 Programm zur...,BÜNDNIS 90DIE GRÜNEN treten zu,[],[],NaN,"['BUeNDNIS', '90DIE', 'GRUeNEN', '41113.1994',...","['BÜNDNIS', '90DIE', 'GRÜNEN', 'treten', 'zu']"
559,2583,2583,1144,au_pr_el_13_111082.txt,Text-ID: at_pr_el_13_111082.txt\n\nPress Relea...,pagr,1.424854e+09,1.424855e+09,Wed Feb 25 09:46:41 2015,1150,...,2013,Hinweisen auf Vorfälle im Apotheken-Bereich mu...,Datenmissbrauch-Skandal: Lugar fordert soforti...,Hinweisen auf Vorfälle im Apotheken-Bereich mu...,Datenmissbrauch-Skandal : Lugar fordert sofort...,[],[],NaN,"['Hinweisen', 'auf', 'Vorfälle', 'im', 'Apothe...","['Datenmissbrauch-Skandal', ':', 'Lugar', 'for..."
560,3421,3421,2175,cd_tw_db_13063_00032.txt,Text ID:\ncd_tw_db_13063_00032.txt\n\n\nTweet ...,kasa,1.410598e+09,1.410599e+09,Sat Sep 13 10:

In [ ]:
non_train.sample_type
